In [1]:
import sys
sys.path.append("/home/tagore/repos/ai/scripts")
import os
import time
import ollama
import chromadb
import torch
import h5py
from torch  import cuda
from transformers import AutoTokenizer, AutoModel
import generate_embeddings as ge

In [2]:
documents_directory = '/home/tagore/repos/ai/data/processed/texts/Bookshelf_NBK279690.pdf'
collection_name = "blast_db_unprocessed"
database_file = '/home/tagore/repos/ai/data/processed/embeddings/chrome_db_unprocessed'

In [3]:
client = chromadb.PersistentClient(path=database_file)
collection = client.get_collection(name=collection_name)

In [4]:
blast_dna_questions = ["What is word size parameter in BLAST?",
"How to get the results of BLASTP in XML format?",
"How to perform a BLAST on a specific taxonomic group?",
"What parameters do I use to perform BLAST with epitopes smaller than 10 amino acids?",
"Which kind of databases can be searched with BLASTX?"]

In [5]:
# an example prompt
prompt = blast_dna_questions[4]

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=3
)
data = results['documents'][0]
data

['Functionality offered by BLAST+ applications\nTom Madden1\nCreated: June 23, 2008; Updated: September 25, 2020.\nThe functionality offered  by the BLAST+ applications has been organized by program type, as to more closely \nresemble Web BLAST.\nAs an example, to run a search of a nucleotide query (translated “on the fly” by BLAST) against a protein \ndatabase one would use the blastx application. The blastx application will also work in “Blast2Sequences” mode \n(i.e.: accept FASTA sequences instead of a BLAST database as targets) and can also send BLAST searches over \nthe network to the public NCBI server if desired.\nThe BLAST+ package offers  three categories of applications: 1.) search tools, 2.) BLAST database tools, and 3.) \nsequence filtering  tools. The blastn, blastp, blastx, tblastx, tblastn, psiblast, rpsblast, and rpstblastn are considered \nsearch applications, as they execute a BLAST search, whereas makeblastdb, blastdb_aliastool, makeprofiledb,  and \nblastdbcmd are c

In [9]:
def generate_prompt(question, context):
        template = """You need to answer questions about specific software.
        Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
        keep the answer concise.
        user
        Question: {question} 
        Context: {context} 
        Do not say according to the text. just give the answer, no comment."""
        return template.format(question=question, context=context)

# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama3",
  prompt=generate_prompt(prompt, data)
)

print(output['response'])

Protein databases can be searched with BLASTX.


## Record the answer after we have add URL data into the database

#### Q: What is word size parameter in BLAST?

A: 
The word size parameter in BLAST is an integer that represents the length of initial exact match. Its default value varies depending on the task:

* megablast: 28
* dc-megablast: 11
* blastn: 11
* blastn-short: 7

#### Q:How to get the results of BLASTP in XML format?

A:
To get the results of BLASTP in XML format, use:

$ blast_formatter –rid X3R7GAUS014 –out test.xml –outfmt "XML"


#### Q: How to perform a BLAST on a specific taxonomic group?

A:
To perform a BLAST on a specific taxonomic group, you can use the following command:

`blastn –db nt –query QUERY –taxids 9606 –outfmt 7 –out OUTPUT.tab`

Replace `QUERY` with your query sequence file, `OUTPUT.tab` with your desired output file name, and `9606` with the taxonomy ID(s) of the specific taxonomic group you want to search.

#### Q:What parameters do I use to perform BLAST with epitopes smaller than 10 amino acids?

A:
To perform BLAST with epitopes smaller than 10 amino acids, you can use the "word_size" parameter and set it to a value between 2-7. The default value is 3 for standard protein-protein comparisons. 

For shorter query sequences, you may want to consider using the "blastp-short" task or increasing the word size to capture smaller epitopes.

#### Q:Which kind of databases can be searched with BLASTX?

A:
Protein databases can be searched with BLASTX.

In [26]:
model_name="dmis-lab/biobert-base-cased-v1.2"
# Initialize BioBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
em = ge.get_embeddings(prompt[4], model, tokenizer)
results = collection3.query(
  query_embeddings=em.tolist(),
  n_results=3
)

NameError: name 'collection3' is not defined